<a href="https://colab.research.google.com/github/redjules/sentiment-analysis/blob/main/Sentiment-cafe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**1. Install and Import Dependencies**



In [41]:
!pip3 install torch torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [42]:
!pip install transformers requests beautifulsoup4 pandas numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [43]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

##**2. Instantiate Model**

In [44]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

##**3. Encode and Calculate Sentiment**

In [45]:
tokens = tokenizer.encode('Meh, it was ok', return_tensors='pt')

In [46]:
#tokens[0]

In [47]:
#tokenizer.decode(tokens[0])

In [48]:
result = model(tokens)

In [49]:
result.logits

tensor([[-0.7896,  1.6718,  2.7634, -0.2691, -2.8158]],
       grad_fn=<AddmmBackward0>)

In [50]:
int(torch.argmax(result.logits))+1

3

##**4. Collect Reviews**

In [51]:
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup = BeautifulSoup(r.text,'html.parser')
regex = re.compile('.*comment*')
results = soup.find_all('p',{'class':regex})
reviews = [result.text for result in results]

In [52]:
reviews

['Great food amazing coffee and tea. Short walk from the harbor. Staff was very friendly',
 "It was ok. Had coffee with my friends. I'm new in the area, still need to discover new places.",
 'Great staff and food. \xa0Must try is the pan fried Gnocchi! \xa0The staff were really friendly and the coffee was good as well',
 "Ricotta hot cakes! These were so yummy. I ate them pretty fast and didn't share with anyone because they were that good ;). I ordered a green smoothie to balance it all out. Smoothie was a nice way to end my brekkie at this restaurant. Others with me ordered the salmon Benedict and the smoked salmon flatbread. They were all delicious and all plates were empty. Cheers!",
 'I came to Social brew cafe for brunch while exploring the city and on my way to the aquarium. I sat outside. The service was great and the food was good too!I ordered smoked salmon, truffle fries, black coffee and beer.',
 "We came for brunch twice in our week-long visit to Sydney. Everything on the 

##**5. Load Reviews into DataFrame and Score**

In [53]:
import numpy as np
import pandas as pd

In [54]:
df = pd.DataFrame(np.array(reviews), 
                   columns=['review'])

In [55]:
df['review'].iloc[0]

'Great food amazing coffee and tea. Short walk from the harbor. Staff was very friendly'

In [56]:
def sentiment_score(review):
  tokens = tokenizer.encode(review, return_tensors='pt')
  result = model(tokens)
  return int(torch.argmax(result.logits))+1

In [57]:
sentiment_score(df['review'].iloc[1])




3

In [58]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [59]:
df

,review,sentiment
0,Great food amazing coffee and tea. Short walk ...,5
1,It was ok. Had coffee with my friends. I'm new...,3
2,Great staff and food. Must try is the pan fri...,5
3,Ricotta hot cakes! These were so yummy. I ate ...,5
4,I came to Social brew cafe for brunch while ex...,5
5,We came for brunch twice in our week-long visi...,4
6,It was ok. The coffee wasn't the best but it w...,3
7,I went here a little while ago- a beautiful mo...,2
8,Great coffee and vibe. That's all you need. C...,5
9,Great coffee and vibe. That's all you need. C...,4


In [60]:
df['review'].iloc[3]

"Ricotta hot cakes! These were so yummy. I ate them pretty fast and didn't share with anyone because they were that good ;). I ordered a green smoothie to balance it all out. Smoothie was a nice way to end my brekkie at this restaurant. Others with me ordered the salmon Benedict and the smoked salmon flatbread. They were all delicious and all plates were empty. Cheers!"